In [1]:
import numpy
import spacy

print(f"Numpy version: {numpy.__version__}")
nlp = spacy.load("en_core_web_sm")
print("Spacy model loaded successfully!")

Numpy version: 1.23.5
Spacy model loaded successfully!


In [3]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_price_range_dynamic(user_input):
    """
    Dynamically extract price range from user input without relying heavily on specific keywords.
    """
    doc = nlp(user_input)
    price_range = {"MinPrice": None, "MaxPrice": None}

    # Variables to help interpret numeric context
    lower_bound = None
    upper_bound = None
    range_detected = False

    # Iterate through tokens to understand price-related contexts
    for token in doc:
        # Check if token is a valid number or MONEY-related entity
        if token.like_num or token.ent_type_ == "MONEY":
            # Attempt to convert token to a numeric value
            try:
                value = float(token.text.replace("$", "").replace(",", "").replace("k", "000").strip())
            except ValueError:
                continue  # Skip tokens that cannot be converted to numbers
            
            # Check the context around the number
            if token.head.text in ["under", "below", "less"]:
                upper_bound = value
            elif token.head.text in ["over", "above", "more"]:
                lower_bound = value
            elif token.head.text == "between" or token.head.text == "from":
                # Handle range start
                lower_bound = value
                range_detected = True
            elif range_detected and token.dep_ in ["conj", "cc"]:  # Handle range end
                upper_bound = value

        # Interpret qualitative terms (e.g., "cheap", "luxury") dynamically
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000  # Example threshold for cheap range
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000  # Example threshold for luxury range

    # Set the price range
    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound

    return price_range

# Test cases
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
]

for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)


Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': None, 'MaxPrice': None}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costing more t

In [17]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_price_range_dynamic(user_input):
    """
    Extract price range dynamically from user input.
    """
    doc = nlp(user_input)
    price_range = {"MinPrice": None, "MaxPrice": None}

    def parse_number(text):
        """
        Parse numbers from text, handling cases like "$20k", "20 000", or "50000$".
        """
        try:
            text = text.lower().replace("k", "000").replace("$", "").replace(",", "").replace(" ", "").strip("?.!")
            return float(text)
        except ValueError:
            return None

    lower_bound = None
    upper_bound = None
    in_range = False  # To track "between" or "to" ranges
    last_number = None

    for token in doc:
        value = parse_number(token.text)

        # If a numeric value is detected
        if value is not None:
            last_number = value
            if in_range:
                if lower_bound is None:
                    lower_bound = value
                else:
                    upper_bound = value
                in_range = False
            elif token.head.text in ["under", "below", "less", "than"]:
                upper_bound = value
            elif token.head.text in ["over", "above", "more", "than"]:
                lower_bound = value
            elif token.head.text in ["between", "to", "and"] or token.text in ["to", "and"]:
                in_range = True
            elif token.head.text in ["for", "within", "around"] and upper_bound is None:
                upper_bound = value

        # Handle qualitative terms like "cheap" or "luxury"
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000

    # Ensure logical consistency
    if lower_bound and upper_bound and lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound

    # Assign final bounds
    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound
    return price_range

# Test cases
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
    "Can I get something for $40,000?",
    "Is $20k to $50k within my range?",
    "I am looking for a car that costs less than 50000$?",
    "My budget is 20,000$",
    "I prefer below 15,000",
]

for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)


Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': None, 'MaxPrice': None}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 15000.0}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costing more

In [16]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_price_range_dynamic(user_input):
    doc = nlp(user_input)
    price_range = {"MinPrice": None, "MaxPrice": None}

    def parse_number(text):
        """
        Parse numbers from text, handling cases like "$20k", "20 000", or "50000$".
        """
        try:
            text = text.lower().replace("k", "000").replace("$", "").replace(",", "").replace(" ", "").strip("?.!")
            return float(text)
        except ValueError:
            return None

    lower_bound = None
    upper_bound = None
    in_range = False  # Tracks if we're inside a range context

    for token in doc:
        value = parse_number(token.text)

        if value is not None:  # Numeric or money-related token
            head = token.head.text.lower()
            grandparent = token.head.head.text.lower()

            # Case 1: Ranges (e.g., "between $20,000 and $50,000")
            if in_range:
                if lower_bound is None:
                    lower_bound = value
                else:
                    upper_bound = value
                in_range = False  # Reset range flag

            # Case 2: Contextual comparison
            elif head == "than" and grandparent == "more":
                lower_bound = value
            elif head == "than" and grandparent == "less":
                upper_bound = value
            elif head in ["under", "below", "less"]:
                upper_bound = value
            elif head in ["over", "above", "more"]:
                lower_bound = value

            # Case 3: Standalone "to", "and" in ranges
            elif head in ["between", "to", "and"] or token.text in ["to", "and"]:
                in_range = True

            # Case 4: Default assignment (e.g., "for $30,000")
            elif head in ["for", "within", "around"] and upper_bound is None:
                upper_bound = value

        # Case 5: Handle qualitative terms like "cheap" or "luxury"
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000

    # Ensure logical consistency
    if lower_bound and upper_bound and lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound

    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound
    return price_range

# Test cases
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
    "Can I get something for $40,000?",
    "Is $20k to $50k within my range?",
    "I am looking for a car that costs less than 50000$?",
    "My budget is 20,000$",
    "I prefer below 15,000",
    "Show me cars costing between $25,000 and $30,000.",
    "Cars over $50k and below $100k are my preference.",
]

for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)


Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': None, 'MaxPrice': None}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 15000.0}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costing more

In [20]:
import spacy
from spacy.tokens import Span

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def merge_currency(doc):
    """
    Merge currency symbols like $ with adjacent numbers into a single token.
    """
    with doc.retokenize() as retokenizer:
        spans = []
        for i, token in enumerate(doc[:-1]):  # Avoid index out of bounds
            if token.text in ["$", "€", "£"] and doc[i + 1].like_num:
                span = doc[token.i : token.i + 2]  # Create span for "$ 20,000"
                spans.append(span)
        for span in spans:
            retokenizer.merge(span)

def extract_price_range_dynamic(user_input):
    doc = nlp(user_input)
    merge_currency(doc)  # Merge currency symbols and numbers

    price_range = {"MinPrice": None, "MaxPrice": None}

    def parse_number(text):
        """
        Parse numbers from text, handling cases like "$20k", "20 000", or "50000$".
        """
        try:
            text = text.lower().replace("k", "000").replace("$", "").replace(",", "").replace(" ", "").strip("?.!")
            return float(text)
        except ValueError:
            return None

    lower_bound = None
    upper_bound = None
    in_range = False  # Tracks if we're inside a range context
    next_value_as_min = False  # Tracks if the next value should be assigned as MinPrice

    for token in doc:
        value = parse_number(token.text)

        # Explicit handling for "between" ranges
        if token.text.lower() == "between":
            in_range = True
            continue

        if value is not None:  # Numeric or money-related token
            head = token.head.text.lower()
            grandparent = token.head.head.text.lower()

            # Case 1: Ranges triggered by "between" + "and/to"
            if in_range:
                if lower_bound is None:
                    lower_bound = value
                elif upper_bound is None:
                    upper_bound = value
                    in_range = False  # Reset range flag
                continue

            # Case 2: Handle direct contextual comparison
            if head in ["than"]:
                if token.head.head.text.lower() in ["more"]:
                    lower_bound = value
                elif token.head.head.text.lower() in ["less"]:
                    upper_bound = value
            elif head in ["under", "below", "less"]:
                upper_bound = value
            elif head in ["over", "above", "more"]:
                lower_bound = value

            # Case 3: Standalone "to", "and" in ranges
            elif token.text in ["to", "and"]:
                next_value_as_min = True
            elif next_value_as_min:
                lower_bound = value
                next_value_as_min = False

            # Case 4: Default assignment (e.g., "for $30,000")
            elif head in ["for", "within", "around"]:
                upper_bound = value

        # Case 5: Handle qualitative terms like "cheap" or "luxury"
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000

    # Ensure logical consistency
    if lower_bound and upper_bound and lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound

    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound
    return price_range

# Test cases
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
    "Can I get something for $40,000?",
    "Is $20k to $50k within my range?",
    "I am looking for a car that costs less than 50000$?",
    "My budget is 20,000$",
    "I prefer below 15,000",
    "Show me cars costing between $25,000 and $30,000.",
    "Cars over $50k and below $100k are my preference.",
]

for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)


Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': 20000.0, 'MaxPrice': 50000.0}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 15000.0}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costin

In [23]:
import spacy
from spacy.tokens import Span

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def merge_currency(doc):
    """
    Merge currency symbols like $ with adjacent numbers into a single token.
    """
    with doc.retokenize() as retokenizer:
        spans = []
        for i, token in enumerate(doc[:-1]):  # Avoid index out of bounds
            if token.text in ["$", "€", "£"] and doc[i + 1].like_num:
                span = doc[token.i : token.i + 2]  # Create span for "$ 20,000"
                spans.append(span)
        for span in spans:
            retokenizer.merge(span)

def extract_price_range_dynamic(user_input):
    doc = nlp(user_input)
    merge_currency(doc)  # Merge currency symbols and numbers

    price_range = {"MinPrice": None, "MaxPrice": None}

    def parse_number(text):
        """
        Parse numbers from text, handling cases like "$20k", "20 000", or "50000$".
        """
        try:
            text = text.lower().replace("k", "000").replace("$", "").replace(",", "").replace(" ", "").strip("?.!")
            return float(text)
        except ValueError:
            return None

    lower_bound = None
    upper_bound = None
    in_range = False  # Tracks if we're inside a range context

    for token in doc:
        value = parse_number(token.text)

        # Explicit handling for "between" ranges
        if token.text.lower() == "between":
            in_range = True
            continue

        if value is not None:  # Numeric or money-related token
            head = token.head.text.lower()
            grandparent = token.head.head.text.lower()

            # Case 1: Ranges triggered by "between" + "and/to"
            if in_range:
                if lower_bound is None:
                    lower_bound = value
                elif upper_bound is None:
                    upper_bound = value
                    in_range = False  # Reset range flag
                continue

            # Case 2: Handle direct contextual comparison
            if head in ["than"]:
                if token.head.head.text.lower() in ["more", "above"]:
                    lower_bound = value
                elif token.head.head.text.lower() in ["less", "below"]:
                    upper_bound = value
            elif head in ["under", "below", "less"]:
                upper_bound = value
            elif head in ["over", "above", "more"]:
                lower_bound = value

            # Case 3: Standalone "to", "and" in ranges
            elif token.text in ["to", "and"]:
                if lower_bound is None:
                    lower_bound = value
                else:
                    upper_bound = value

            # Case 4: Default assignment (e.g., "for $30,000")
            elif head in ["for", "within", "around"]:
                upper_bound = value

        # Case 5: Handle qualitative terms like "cheap" or "luxury"
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000

    # Ensure logical consistency
    if lower_bound and upper_bound and lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound

    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound
    return price_range

# Test cases
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
    "Can I get something for $40,000?",
    "Is $20k to $50k within my range?",
    "I am looking for a car that costs less than 50000$?",
    "My budget is 20,000$",
    "I prefer below 15,000",
    "Show me cars costing between $25,000 and $30,000.",
    "Cars over $50k and below $100k are my preference.",
]

for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)


Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': 20000.0, 'MaxPrice': 50000.0}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 15000.0}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costin

In [32]:
def extract_price_range_dynamic(user_input):
    doc = nlp(user_input)
    merge_currency(doc)

    price_range = {"MinPrice": None, "MaxPrice": None}

    def parse_number(text):
        try:
            text = text.lower().replace("k", "000").replace("$", "").replace(",", "").replace(" ", "").strip("?.!")
            return float(text)
        except ValueError:
            return None

    lower_bound = None
    upper_bound = None
    in_range = False
    single_number_encountered = False  # Flag for single numbers without context

    for token in doc:
        value = parse_number(token.text)

        # Handle range keywords like "between"
        if token.text.lower() == "between":
            in_range = True
            continue

        if value is not None:  # Numeric token
            # Check immediate head and ancestors
            head = token.head.text.lower()
            ancestors = [ancestor.text.lower() for ancestor in token.ancestors]
            children = [child.text.lower() for child in token.children]  # Include children for better context

            # Handle "more than", "less than" by looking at ancestors and children
            if ("more" in ancestors or "above" in ancestors or "over" in ancestors) and "than" in ancestors:
                lower_bound = value
            elif ("less" in ancestors or "below" in ancestors or "under" in ancestors) and "than" in ancestors:
                upper_bound = value

            # Handle explicit range
            elif in_range:
                if lower_bound is None:
                    lower_bound = value
                else:
                    upper_bound = value
                    in_range = False

            # Handle standalone comparison keywords
            elif head in ["under", "below", "less"]:
                upper_bound = value
            elif head in ["over", "above", "more"]:
                lower_bound = value

            # Handle ranges connected by "to", "and"
            elif token.text in ["to", "and"]:
                if lower_bound is None:
                    lower_bound = value
                else:
                    upper_bound = value

            # Default catch for exact values
            elif head in ["for", "within", "around"]:
                upper_bound = value

            # If no context is found, treat the number as an upper bound by default
            else:
                single_number_encountered = True
                upper_bound = value

        # Handle qualitative terms like "cheap" or "luxury"
        elif token.lemma_ in ["cheap", "affordable"]:
            upper_bound = 20000
        elif token.lemma_ in ["luxury", "expensive"]:
            lower_bound = 50000

    # Swap bounds if lower > upper
    if lower_bound and upper_bound and lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound

    # If a single number was encountered without context, assign it as an upper bound
    if single_number_encountered and lower_bound is None and upper_bound is None:
        upper_bound = value

    price_range["MinPrice"] = lower_bound
    price_range["MaxPrice"] = upper_bound
    return price_range


In [34]:
examples = [
    "I'm looking for a car under $40,000.",
    "Do you have anything between $20,000 and $50,000?",
    "I want an affordable car.",
    "Show me luxury cars over $70,000.",
    "Give me a cheap vehicle under $15k.",
    "I need a car below $25,000 but above $10,000.",
    "I’m searching for something costing more than 50k.",
    "What can I get for less than $30,000?",
    "Can I get something for $40,000?",
    "Is $20k to $50k within my range?",
    "I am looking for a car that costs less than 50000$?",
    "My budget is 20,000$",
    "I prefer below 15,000",
    "Show me cars costing between $25,000 and $30,000.",
    "Cars over $50k and below $100k are my preference.",
]
for example in examples:
    result = extract_price_range_dynamic(example)
    print(f"Input: {example}")
    print(f"Extracted Price Range: {result}")
    print("-" * 50)

Input: I'm looking for a car under $40,000.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 40000.0}
--------------------------------------------------
Input: Do you have anything between $20,000 and $50,000?
Extracted Price Range: {'MinPrice': 20000.0, 'MaxPrice': 50000.0}
--------------------------------------------------
Input: I want an affordable car.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 20000}
--------------------------------------------------
Input: Show me luxury cars over $70,000.
Extracted Price Range: {'MinPrice': 70000.0, 'MaxPrice': None}
--------------------------------------------------
Input: Give me a cheap vehicle under $15k.
Extracted Price Range: {'MinPrice': None, 'MaxPrice': 15000.0}
--------------------------------------------------
Input: I need a car below $25,000 but above $10,000.
Extracted Price Range: {'MinPrice': 10000.0, 'MaxPrice': 25000.0}
--------------------------------------------------
Input: I’m searching for something costin

In [47]:
def extract_colors(user_input):
    # List of unique colors
    UNIQUE_COLORS = ['White', 'Gray', 'Black', 'Silver', 'Red', 'Blue', 
                     'Green', 'Gold', 'Copper', 'Brown', 'Orange', 'Tan', 
                     'Teal', 'Yellow', 'Maroon']

    # Define darker and lighter color groups
    DARK_COLORS = ['Black', 'Gray', 'Navy', 'Charcoal', 'Brown', 'Maroon', 'Dark Green']
    LIGHT_COLORS = ['White', 'Silver', 'Yellow', 'Beige', 'Light Gray', 'Light Blue', 'Pale Orange']

    # Synonyms for tone-related words
    TONE_MAP = {
        "dark": "dark",
        "light": "light",
        "bright": "light",
        "deep": "dark",
        "pale": "light",
    }

    # Synonyms for colors
    COLOR_SYNONYMS = {
        "grey": "Gray",
        "charcoal": "Black",
        "scarlet": "Red",
        "navy": "Blue",
        "emerald": "Green",
        "golden": "Gold",
        "bronze": "Copper",
        "chocolate": "Brown",
        "amber": "Orange",
        "beige": "Tan",
        "turquoise": "Teal",
        "lemon": "Yellow",
        "burgundy": "Maroon",
    }

    # Reverse mapping for quick lookup
    COLOR_MAP = {color.lower(): color for color in UNIQUE_COLORS}
    for synonym, base_color in COLOR_SYNONYMS.items():
        COLOR_MAP[synonym.lower()] = base_color

    doc = nlp(user_input)
    detected_colors = []
    match_dark = False
    match_light = False
    explicit_color_found = False

    for token in doc:
        token_text = token.text.lower()

        # Check for tone keywords ("dark", "light") with broader context
        if token_text in TONE_MAP:
            tone = TONE_MAP[token_text]
            if tone == "dark":
                match_dark = True
            elif tone == "light":
                match_light = True

        # Check if the token matches any color or synonym
        if token_text in COLOR_MAP:
            base_color = COLOR_MAP[token_text]
            detected_colors.append(base_color)
            explicit_color_found = True

        # Handle tone + color combination (e.g., "dark blue")
        if token_text in TONE_MAP:
            for child in token.children:
                child_text = child.text.lower()
                if child_text in COLOR_MAP:
                    base_color = COLOR_MAP[child_text]
                    detected_colors.append(base_color)
                    explicit_color_found = True

    # If no specific color is mentioned but tone is detected, use dark/light groups
    if not explicit_color_found:
        if match_dark:
            detected_colors.extend(DARK_COLORS)
        if match_light:
            detected_colors.extend(LIGHT_COLORS)

    # Remove duplicates and preserve order
    detected_colors = list(dict.fromkeys(detected_colors))

    return detected_colors


In [48]:
examples = [
    "I want a dark blue car.",
    "Do you have something in red or green?",
    "Show me vehicles in navy, charcoal, or turquoise.",
    "I prefer a bright yellow or pale orange car.",
    "Give me a car that's emerald or burgundy.",
    "I'm looking for a car in dark gray or light silver.",
    "Show me darker colors.",
    "Do you have lighter shades?",
    "I want a car in dark or bright colors.",
    "I’m interested in light-colored cars.",
]

for example in examples:
    result = extract_colors(example)
    print(f"Input: {example}")
    print(f"Extracted Colors: {result}")
    print("--------------------------------------------------")


Input: I want a dark blue car.
Extracted Colors: ['Blue']
--------------------------------------------------
Input: Do you have something in red or green?
Extracted Colors: ['Red', 'Green']
--------------------------------------------------
Input: Show me vehicles in navy, charcoal, or turquoise.
Extracted Colors: ['Blue', 'Black', 'Teal']
--------------------------------------------------
Input: I prefer a bright yellow or pale orange car.
Extracted Colors: ['Yellow', 'Orange']
--------------------------------------------------
Input: Give me a car that's emerald or burgundy.
Extracted Colors: ['Green', 'Maroon']
--------------------------------------------------
Input: I'm looking for a car in dark gray or light silver.
Extracted Colors: ['Gray', 'Silver']
--------------------------------------------------
Input: Show me darker colors.
Extracted Colors: []
--------------------------------------------------
Input: Do you have lighter shades?
Extracted Colors: []
----------------------

In [2]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased")

# Example usage:
input_text = "I'm looking for a red car under $40,000."
question = "What color is the user looking for?"
color = qa_pipeline({"context": input_text, "question": question})['answer']

print("Extracted Color:", color)


c:\Users\karim\OneDrive\Desktop\CODEJAM14Repo\CodeJam14KKSB\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\karim\OneDrive\Desktop\CODEJAM14Repo\CodeJam14KKSB\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\karim\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or t

Extracted Color: 40


In [18]:
nlp = spacy.blank("en")
# Add NER pipeline
ner = nlp.add_pipe("ner")

# Add labels
ner.add_label("PRICE")
ner.add_label("COLOR")
ner.add_label("BODY_TYPE")

1

In [28]:

import spacy
from spacy.training import Example
#import kagglehub


# Download latest version
#path1 = kagglehub.dataset_download("lepchenkov/usedcarscatalog")

#print("Path to dataset files:", path1)

# Download latest version
#path = kagglehub.dataset_download("thedevastator/uncovering-factors-that-affect-used-car-prices")

#print("Path to dataset files:", path)

# Load a blank English model
nlp = spacy.blank("en")

# Add the NER component to the pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")
# Define your training examples
examples = [
    ("I need a green SUV under $20,000.", {"entities": [(9, 14, "COLOR"), (15, 18, "BODY_TYPE"), (19, 32, "PRICE")]}),
    ("Show me a blue sedan for $25,000.", {"entities": [(10, 14, "COLOR"), (15, 20, "BODY_TYPE"), (25, 32, "PRICE")]}),
    ("Can you find me a red 4x4 around $50,000?", {"entities": [(18, 21, "COLOR"), (22, 25, "BODY_TYPE"), (26, 40, "PRICE")]}),
    ("Do you have silver trucks below $30,000?", {"entities": [(12, 18, "COLOR"), (19, 25, "BODY_TYPE"), (27, 40, "PRICE")]}),
    ("I'm interested in luxury SUVs costing $60,000.", {"entities": [(25, 29, "BODY_TYPE"), (38, 45, "PRICE")]}),
    ("Can I buy a car for less than $15,000?", {"entities": [(20, 37, "PRICE")]}),
    ("Find me a white sedan priced under $40,000.", {"entities": [(10, 15, "COLOR"), (16, 21, "BODY_TYPE"), (29, 42, "PRICE")]}),
    ("Are there black SUVs costing above $70,000?", {"entities": [(10, 15, "COLOR"), (16, 20, "BODY_TYPE"), (29, 42, "PRICE")]}),
    ("Can you show me an orange hatchback for $18,000?", {"entities": [(19, 25, "COLOR"), (26, 35, "BODY_TYPE"), (41, 48, "PRICE")]}),
    ("I'm looking for something costing between $25,000 and $50,000.", {"entities": [(34, 40, "PRICE"), (42, 49, "PRICE"), (54, 61, "PRICE")]}),
]

# Add labels to the NER component
for _, annotations in examples:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Disable other pipeline components during training
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

# Begin training
with nlp.disable_pipes(*unaffected_pipes):  # Only train NER
    optimizer = nlp.begin_training()
    for epoch in range(50):  # Number of iterations
        losses = {}
        for text, annotations in examples:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], sgd=optimizer, drop=0.1, losses=losses)
        print(f"Epoch {epoch + 1}, Loss: {losses['ner']}")

# Save the model
nlp.to_disk("custom_ner_model")

c:\Users\karim\OneDrive\Desktop\CODEJAM14Repo\CodeJam14KKSB\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Do you have silver trucks below $30,000?" with entities "[(12, 18, 'COLOR'), (19, 25, 'BODY_TYPE'), (27, 40...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\karim\OneDrive\Desktop\CODEJAM14Repo\CodeJam14KKSB\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'm looking for something costing between $25,000 ..." with entities "[(34, 40, 'PRICE'), (42, 49, 'PRICE'), (54, 61, 'P...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 1, Loss: 78.6543298996985
Epoch 2, Loss: 42.160659491403166
Epoch 3, Loss: 31.77892312898595
Epoch 4, Loss: 31.552385881799147
Epoch 5, Loss: 27.963566404160247
Epoch 6, Loss: 8.92374638860474
Epoch 7, Loss: 4.766545173435727
Epoch 8, Loss: 1.9822155046019756
Epoch 9, Loss: 0.004723388709847289
Epoch 10, Loss: 0.00014075961142296959
Epoch 11, Loss: 0.0002687481371512938
Epoch 12, Loss: 9.565561938704684e-06
Epoch 13, Loss: 6.0878744757416635e-05
Epoch 14, Loss: 9.566397447421282e-06
Epoch 15, Loss: 3.968191136314691e-06
Epoch 16, Loss: 4.435601069806875e-07
Epoch 17, Loss: 7.656302840084736e-07
Epoch 18, Loss: 2.495474556429283e-07
Epoch 19, Loss: 4.310668368491009e-07
Epoch 20, Loss: 3.599860791242027e-07
Epoch 21, Loss: 2.842558257211898e-06
Epoch 22, Loss: 5.146057818906265e-05
Epoch 23, Loss: 7.334826216603394e-07
Epoch 24, Loss: 2.1547119676925475e-06
Epoch 25, Loss: 6.013236337460673e-07
Epoch 26, Loss: 2.553902447811143e-06
Epoch 27, Loss: 3.2384342072195402e-06
Epoch 28, 

In [30]:


import spacy

# Load the trained model
nlp = spacy.load("custom_ner_model")

# Define test cases (only Q parts are used for entity extraction)
test_cases = [
    "I need a pickup truck with 4WD, leather seats, and under 50,000 km.",
    "Do you have a sporty car with a sunroof and under 50,000 km?",
    "Do you have a red convertible under 30,000 km?",
    "Do you have a family SUV with leather seats and low mileage?",
    "Do you have an electric car with a range of at least 300 miles and advanced safety features?",
    "I'm looking for an SUV with captain's chairs and a panoramic sunroof.",
    "Do you have a minivan with stow-and-go seating and rear-seat entertainment?",
    "I'm looking for a high-performance car with launch control and Brembo brakes.",
    "Do you have the car with VIN 1HGCM82633A123456?",
    "I'm looking for the car with VIN 5FNYF6H57LB789012.",
    "Do you have an SUV with off-road capabilities, a roof rack, and skid plates?",
    "Do you have an SUV with remote start, a heated steering wheel, and a power liftgate?",
    "I'm looking for an SUV with third-row seating.",
    "Do you have a hybrid SUV?",
    "I want something fun to drive.",
    "I currently drive a 2018 Toyota Camry. Do you have anything similar?",
    "I'm driving a 2018 Hyundai Tucson right now. Any similar SUVs in stock?",
    "I'm looking for a fun car to drive.",
    "I want an electric car.",
]

# Test the model with the cases
for i, sentence in enumerate(test_cases, start=1):
    doc = nlp(sentence)
    print(f"Test Case {i}:")
    print(f"Sentence: {sentence}")
    print("Entities:")
    for ent in doc.ents:
        print(f" - {ent.text}: {ent.label_}")
    print("\n" + "-"*50 + "\n")



Test Case 1:
Sentence: I need a pickup truck with 4WD, leather seats, and under 50,000 km.
Entities:
 - pickup: COLOR
 - with 4WD: PRICE
 - , leather seats,: PRICE
 - under 50,000: PRICE

--------------------------------------------------

Test Case 2:
Sentence: Do you have a sporty car with a sunroof and under 50,000 km?
Entities:
 - sporty: COLOR
 - under 50,000: PRICE

--------------------------------------------------

Test Case 3:
Sentence: Do you have a red convertible under 30,000 km?
Entities:
 - red: COLOR
 - under 30,000: PRICE

--------------------------------------------------

Test Case 4:
Sentence: Do you have a family SUV with leather seats and low mileage?
Entities:
 - family: COLOR
 - SUV: BODY_TYPE
 - with leather seats and: PRICE
 - mileage?: PRICE

--------------------------------------------------

Test Case 5:
Sentence: Do you have an electric car with a range of at least 300 miles and advanced safety features?
Entities:
 - range: COLOR
 - of: BODY_TYPE
 - safety: